In [1]:
!rm -rf logs/
!rm ngrok*
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2018-08-17 06:41:26--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.5.182.176, 52.44.144.199, 52.203.66.95, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.5.182.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  22.5MB/s    in 0.2s    

2018-08-17 06:41:26 (22.5 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
LOG_DIR = './logs'
get_ipython().system_raw('tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'.format(LOG_DIR))

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [4]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://46824287.ngrok.io


In [0]:
import tensorflow as tf
import numpy as np

from sklearn.datasets import load_iris

In [0]:
x = load_iris().data
y = load_iris().target

In [7]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
y = to_categorical(y)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.20)

In [1]:
tf.reset_default_graph()

with tf.name_scope('input'):
  input_features = tf.placeholder(dtype=tf.float32, shape=[None, x.shape[1]], name='input_features')
  input_labels = tf.placeholder(dtype=tf.float32, shape=[None, y.shape[1]], name='input_labels')
  
with tf.name_scope('model'):
  #Logistic Regression
  #weights = tf.Variable(tf.random_normal(shape=[x.shape[1], y.shape[1]]), name='weights')
  #biases = tf.Variable(tf.random_normal(shape=[y.shape[1]]))
  #linear_model = tf.add(tf.matmul(input_features, weights), biases)
  #predictions = tf.nn.softmax(linear_model)
  
  #Neural Network
  weights = {
      'hidden_layer': tf.Variable(tf.random_normal(shape=[x.shape[1], 16]), name='weights'),
      'output_layer': tf.Variable(tf.random_normal(shape=[16, y.shape[1]]), name='weights')
  }
  biases = {
      'hidden_layer': tf.Variable(tf.random_normal(shape=[16])),
      'output_layer': tf.Variable(tf.random_normal(shape=[y.shape[1]]))
  }
  hidden_layer = tf.add(tf.matmul(input_features, weights['hidden_layer']), biases['hidden_layer'])
  hidden_layer = tf.nn.relu(hidden_layer)
  output_layer = tf.add(tf.matmul(hidden_layer, weights['output_layer']), biases['output_layer'])
  predictions = tf.nn.softmax(output_layer)
  

  #Optimizer
with tf.name_scope('training_ops'):
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=output_layer, labels=input_labels))
  train_op = tf.train.AdamOptimizer(learning_rate=1e-2).minimize(loss)
  tf.summary.scalar(name='loss', tensor=loss)
  
with tf.name_scope('metrics'):
  correct_prediction = tf.cast(tf.equal(tf.argmax(predictions, 1), tf.argmax(input_labels, 1)), tf.float32)
  accuracy_op = tf.reduce_mean(correct_prediction)
  tf.summary.scalar(name='accuracy', tensor=accuracy_op)

  summary = tf.summary.merge_all()
  writer = tf.summary.FileWriter(logdir='./logs', graph=tf.get_default_graph())

NameError: ignored

In [0]:
init_op = tf.global_variables_initializer()

In [0]:
def next_batch(batch_size, features, labels):
  indices = np.arange(start=0, stop=features.shape[0])
  np.random.shuffle(indices)
  indices = indices[:batch_size]
  return features[indices], labels[indices]

In [17]:
with tf.Session() as sess:
  sess.run(init_op)
  for epoch in range(10):
    for mini_batch in range(int(x_train.shape[0] / 16)):
      batch_x, batch_y = next_batch(batch_size=16, 
                                              features=x_train, 
                                              labels=y_train)
      _, train_loss, train_accuracy, train_summary = sess.run([train_op, loss, accuracy_op, summary], 
                                               feed_dict={
                                                   input_features: batch_x,
                                                   input_labels: batch_y
                                               })
      writer.add_summary(summary=train_summary, global_step=mini_batch)
    print('Epoch {}, loss: {}, accuracy: {}'.format(epoch, train_loss, train_accuracy))

Epoch 0, loss: 7.668201446533203, accuracy: 0.5
Epoch 1, loss: 4.223006248474121, accuracy: 0.125
Epoch 2, loss: 3.9749138355255127, accuracy: 0.0
Epoch 3, loss: 2.567575216293335, accuracy: 0.1875
Epoch 4, loss: 1.6119773387908936, accuracy: 0.25
Epoch 5, loss: 1.5585500001907349, accuracy: 0.1875
Epoch 6, loss: 1.2723591327667236, accuracy: 0.3125
Epoch 7, loss: 0.6243999600410461, accuracy: 0.75
Epoch 8, loss: 0.24057278037071228, accuracy: 0.9375
Epoch 9, loss: 0.32272985577583313, accuracy: 0.9375


In [19]:
with tf.Session() as sess:
  sess.run(init_op)
  prediction_ = sess.run(predictions, feed_dict={input_features: x_test[10].reshape(-1, x_test.shape[1])})
  print(prediction_)
  print('predicted class : {}, true class : {}'.format(sess.run(tf.argmax(prediction_, 1)), sess.run(tf.argmax(y_test[10]))))

[[9.6836042e-01 3.1639565e-02 1.6674779e-14]]
predicted class : [0], true class : 0
